데이터세트, 데이터로더를 활용한 다중선형회귀 구현

In [1]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# 데이터 생성
train_x = torch.FloatTensor([
    [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7]
])
train_y = torch.FloatTensor([
    [0.1, 1.5], [1, 2.8], [1.9, 4.1], [2.8, 5.4], [3.7, 6.7], [4.6, 8]
])

In [3]:
# 텐서 데이터세트를 활용해 훈련용 데이터세트 생성
train_dataset = TensorDataset(train_x, train_y)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=True)

batch_size=2: 한 번의 배치마다 두 개의 데이터 샘플과 정답을 가져오게 함.  
shuffle=True: 불러오는 데이터의 순서를 무작위로 설정  
drop_last=True: 배치 크기에 맞지 않는 배치를 제거  
-> 전체 데이터세트의 크기가 5일 때, 배치 크기가 2라면 마지막 배치의 크기는 1이 된다.  
-> 배치 크기로 설정한 2보다 작으므로 마지막 배치를 학습에 사용하지 않겠다는 것이다.

In [4]:
# 모델, Loss, 옵티마이저 선언
model = nn.Linear(2, 2, bias=True)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

train_x, train_y 모두 (n, 2)의 크기를 가지므로 입/출력 데이터 차원 크기는 모두 2가 된다.

In [5]:
for epoch in range(20000):
    cost = 0.0

    # 미니 배치 단위로 학습
    for batch in train_dataloader:
        x, y = batch # 데이터 분리
        output = model(x) # 예측

        loss = criterion(output, y) # 손실 계산

        optimizer.zero_grad() # 이전 그래디언트 초기화
        loss.backward() # 역전파를 통한 그래디언트 계산
        optimizer.step() # 파라미터 업데이트

        cost += loss # 배치 손실 누적

    # 에포크 당 평균 손실 계산
    cost = cost / len(train_dataloader)

    # 1000 에포크마다 로그 출력
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[0.4803, 0.3181],
        [0.8498, 0.3685]], requires_grad=True), Parameter containing:
tensor([-0.6911,  0.1746], requires_grad=True)], Cost : 0.030
Epoch : 2000, Model : [Parameter containing:
tensor([[0.5833, 0.2649],
        [0.9326, 0.3258]], requires_grad=True), Parameter containing:
tensor([-0.8474,  0.0491], requires_grad=True)], Cost : 0.008
Epoch : 3000, Model : [Parameter containing:
tensor([[0.6358, 0.2378],
        [0.9748, 0.3040]], requires_grad=True), Parameter containing:
tensor([-0.9270, -0.0149], requires_grad=True)], Cost : 0.002
Epoch : 4000, Model : [Parameter containing:
tensor([[0.6626, 0.2240],
        [0.9963, 0.2929]], requires_grad=True), Parameter containing:
tensor([-0.9675, -0.0475], requires_grad=True)], Cost : 0.001
Epoch : 5000, Model : [Parameter containing:
tensor([[0.6762, 0.2170],
        [1.0073, 0.2872]], requires_grad=True), Parameter containing:
tensor([-0.9882, -0.0641], requires_grad=True)]

### 📖 TensorDataset: 데이터 포장하기
TensorDataset은 여러 텐서를 마치 하나의 **'묶음 상품'**처럼 만들어주는 역할을 합니다.  

핵심 역할: 입력 데이터(Features)와 정답 데이터(Labels)를 한 쌍으로 묶어줍니다.  

동작 방식: 인덱스(index)를 통해 특정 위치의 데이터 쌍([입력, 정답])에 접근할 수 있게 합니다.  

예시: my_dataset[0] → 첫 번째 [입력, 정답] 쌍을 반환  

비유: 여러 재료(입력, 정답)를 하나의 밀키트(Meal Kit) 안에 가지런히 담아두는 것과 같습니다.  

### 👨‍🍳 DataLoader: 데이터 공급하기
DataLoader는 포장된 데이터셋(TensorDataset)을 받아, 모델이 학습할 수 있도록 효율적으로 '서빙'해주는 역할을 합니다.  

핵심 역할: 데이터셋을 미니배치(mini-batch) 단위로 나누어 전달합니다.  

주요 기능:  

batch_size: 한 번에 모델에 전달할 데이터의 양(묶음 크기)을 결정합니다.

shuffle: 데이터 순서를 무작위로 섞어 모델의 과적합을 방지합니다.

num_workers: 여러 프로세스를 사용해 데이터 로딩 속도를 높입니다.

비유: 밀키트(TensorDataset)를 받아, 레시피에 맞게 정해진 양만큼 재료를 꺼내주는 요리사와 같습니다.

### ⚙️ 둘의 관계: 학습 워크플로우
모델 학습 시 이 둘은 다음과 같은 순서로 함께 사용됩니다.

데이터 준비:
입력(x)과 정답(y) 데이터를 torch.Tensor로 만듭니다.  

포장 (TensorDataset):
dataset = TensorDataset(x, y)  
입력과 정답 텐서를 하나의 데이터셋으로 묶습니다.

서빙 준비 (DataLoader):  
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)  
묶인 데이터셋을 32개씩, 순서를 섞어서 전달하도록 설정합니다.

모델 학습:  
for inputs, labels in dataloader:  
DataLoader가 제공하는 미니배치를 사용하여 모델을 학습시킵니다.